In [1]:
import pandas as pd

from IPython.display import HTML

In [2]:
dh = pd.read_csv('sequencias.csv')

In [3]:
dh

,demanda,sequencia,so_paras
0,38,NaN,FOOa-s
1,73,NaN,FOOOaOaa-c-s
2,74,NaN,FOOOOOOOa-s
3,77,NaN,Oaa-c-s
4,78,NaN,Oaa-c-s
...,...,...,...
24733,25498,NaN,I-S
24734,25500,NaN,I-S
24735,25504,NaN,I-S
24736,25507,NaN,-x


In [4]:
sequencia = 'so_paras'  # 'sequencia'

dh.loc[dh.demanda==38, sequencia].values[0]

'FOOa-s'

In [5]:
dh.nunique()

demanda      24738
sequencia        0
so_paras       480
dtype: int64

In [6]:
dt = dh.groupby([sequencia]).nunique().sort_values(by=['sequencia'], ascending=False)

dt.to_csv('fluxos.csv')

In [7]:
dh.so_paras.map(str)
dh.loc[dh.so_paras.str.contains('F') & dh.so_paras.str.contains('I'), :]

,demanda,sequencia,so_paras
4214,4306,NaN,FaII-s
4343,4435,NaN,FaIF-x
4627,4719,NaN,FaI-s
4723,4815,NaN,FaI-x
4737,4829,NaN,FaI-s
...,...,...,...
20747,20851,NaN,OIFFaOaaaaFa-c
21141,21245,NaN,FFaFIFFaOaaaaFa-c
21203,21307,NaN,FFaFnI-s
21215,21319,NaN,OFnII-S


In [8]:
dh.loc[dh.so_paras.str.contains('A') & dh.so_paras.str.contains('I'), :]

,demanda,sequencia,so_paras
20950,21054,NaN,InAAAAaa
22190,22295,NaN,InAAAAaaaaaa-C
22342,22448,NaN,InAAAAaaaaaa-C
22913,23107,NaN,IInAAAAaaaaaa-C
22915,23111,NaN,IInAAAAaaaaaa-C
23228,23518,NaN,InAAAAaaaaaa-C
23311,23622,NaN,InAAAAaaaaaa-C
23392,23743,NaN,AnI-S
23772,24214,NaN,AI-S
23932,24417,NaN,AnI-S


In [9]:
dh.loc[dh.so_paras.str.contains('F') & dh.so_paras.str.contains('R'), :]

,demanda,sequencia,so_paras


In [10]:
dt1 = pd.read_csv('fluxos.csv')
dt1[sequencia].map(str)

0                    -P
1                   -Pn
2                  OI-s
3           OFnOFFnOn-x
4               OFnII-S
             ...       
475    FFaaaaaaaaaaFa-c
476     FFaaaaaaaaaFa-c
477      FFaaaaaaaaFa-s
478      FFaaaaaaaaFa-c
479                  nR
Name: so_paras, Length: 480, dtype: object

In [ ]:
"""# quanto ao ônus

dt2 = dt1.copy()

com_onus = dt2[sequencia].str.endswith(("Demanda com ônus em pontos de função concluída')",
                                         "Demanda concluída com ônus')"))
sem_onus = dt2[sequencia].str.endswith(("Demanda sem ônus em pontos de função concluída')", 
                                         "Demanda concluída sem ônus')"))

dt2.loc[com_onus, 'onus'] = 'com'
dt2.loc[sem_onus, 'onus'] = 'sem'

# quanto a finalizacao em geral

eh_cancelada = dt2[sequencia].str.endswith("Demanda cancelada')")
eh_dba = dt2[sequencia].str.endswith(( "Concluído')", "Aceito')", "Cancelado')" ))
eh_defeito_ou_pdti = dt2[sequencia].str.endswith(( "Fechado')", "'Iniciado')", "Cancelado')", "Rejeição cancelada')" ))

dt2.loc[eh_cancelada, 'onus'] = 'cancelada'
dt2.loc[eh_dba, 'onus'] = 'dba'
dt2.loc[eh_defeito_ou_pdti, 'onus'] = 'defeito_ou_pdti'"""

In [ ]:
"""eh_fabrica = dt2[sequencia].str.contains("fábrica") | dt2[sequencia].str.contains("artefato") | dt2[sequencia].str.contains("Contar") & (~dt2[sequencia].str.contains("iternamente"))
#(
#    dt2[sequencia].str.contains("Realizar implementação") 
#    | dt2[sequencia].str.contains("Executar manutenção") 
#    | dt2[sequencia].str.contains("Contar pontos de função")
#    & (~dt2[sequencia].str.contains("Desenvolver internamente"))
#)
eh_interna = dt2[sequencia].str.contains("internamente") & (~dt2[sequencia].str.contains("fábrica") & ~dt2[sequencia].str.contains("artefato")) & ~dt2[sequencia].str.contains("Contar")
#(
#    dt2[sequencia].str.contains("Desenvolver internamente") 
#    & ~dt2[sequencia].str.contains("Realizar implementação") 
#    & ~dt2[sequencia].str.contains("Executar manutenção")
#)

#dt2['tipo'] = 'aberta'  ## default 

dt2.loc[eh_fabrica, 'tipo'] = 'fabrica'
dt2.loc[eh_interna, 'tipo'] = 'interna'"""